In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.1'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

2022-11-10 04:24:51.509935: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 04:24:51.515951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 04:24:51.516250: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        # img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA),
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-11-10 04:24:51.764387: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-10 04:24:51.765151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 04:24:51.765392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 04:24:51.765587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 256, 256, 3), dtype=float32, numpy=
array([[[[159.72015  , 150.28152  , 148.81667  ],
         [158.30792  , 148.8693   , 147.40443  ],
         [158.3232   , 148.88457  , 147.41972  ],
         ...,
         [161.32281  , 154.92259  , 157.10382  ],
         [161.30641  , 154.90619  , 157.08743  ],
         [159.92096  , 153.52074  , 155.70197  ]],

        [[157.13528  , 147.69191  , 146.22705  ],
         [158.1836   , 148.74022  , 147.27536  ],
         [157.46635  , 148.02298  , 146.55812  ],
         ...,
         [162.79521  , 156.39499  , 158.57622  ],
         [160.234    , 153.83377  , 156.01501  ],
         [161.05673  , 154.65651  , 156.83774  ]],

        [[156.27933  , 146.53685  , 145.07199  ],
         [156.98195  , 147.23949  , 145.77463  ],
         [155.19955  , 145.45709  , 143.99223  ],
         ...,
         [163.05402  , 156.6538   , 158.83504  ],
         [161.542    , 154.9473   , 157.22577  ],
         [159.92868  , 153.31006  , 155.6005  

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]], dtype=float32)>

### Model

In [27]:
def createModel():
    inp = tf.keras.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

    conv1 = tf.keras.layers.Conv2D(96, 7, strides=(4,4), activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv1')
    pool1 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool1')

    conv2 = tf.keras.layers.Conv2D(256, 5, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv2')
    pool2 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool2')

    conv3 = tf.keras.layers.Conv2D(384, 3, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='conv3')
    pool3 = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=2, name='pool3')

    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = tf.keras.layers.Rescaling(1.0/255.0, name='pre_rescale')(inp)
    o = tf.keras.layers.Normalization(axis=3, name='pre_norm')(o)
    o = conv1(o)
    o = pool1(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [2, 2], [2, 2], [0, 0]])
    o = conv2(o)
    o = pool2(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = tf.pad(o, [[0, 0], [1, 1], [1, 1], [0, 0]])
    o = conv3(o)
    o = pool3(o)
    o = tf.nn.local_response_normalization(o, depth_radius=5, alpha=0.0001, beta=0.75)

    o = flat1(o)
    o = fc1(o)
    o = do1(o)


    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='cnn_10')
    
    return model

In [28]:
model = createModel()

In [29]:
model.summary()

Model: "cnn_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 pre_rescale (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 pre_norm (Normalization)    (None, 256, 256, 3)       7         
                                                                 
 conv1 (Conv2D)              (None, 63, 63, 96)        14208     
                                                                 
 pool1 (MaxPooling2D)        (None, 31, 31, 96)        0         
                                                                 
 tf.nn.local_response_normal  (None, 31, 31, 96)       0         
 ization (TFOpLambda)                                            
                                                            

### Compile Model

In [30]:
STEPS_PER_EPOCH = 1000

EPOCH = 300

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [31]:
def schedule(epoch, lr):
    if(epoch == 10000*50//STEPS_PER_EPOCH):
        return lr * 0.1
    else:
        return lr

In [32]:
logBasePath = "log/CNN_10"
logPrefix = "log"

In [33]:
for i, (train, val) in enumerate([folds[0]]):
    
    model = createModel()
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb, lrSchedule]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-11-10 04:24:54.365293: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-10 04:24:54.365317: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-10 04:24:54.365339: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-10 04:24:54.445791: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2022-11-10 04:24:54.447159: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2022-11-10 04:24:55.001113: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


  54/3528 [..............................] - ETA: 10s - loss: 0.7475 - accuracy: 0.3889  

2022-11-10 04:24:55.793212: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 822.72MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 04:24:55.793345: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 822.72MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 04:24:55.802883: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 843.88MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 04:24:55.802902: W tensorflow/core/common_runtime/bfc_al

3528/3528 [==============================] - 11s 3ms/step - loss: 0.7003 - accuracy: 0.5232
Epoch 1/300


2022-11-10 04:25:05.351712: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 819.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 04:25:05.351752: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 819.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 04:25:05.360519: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 841.75MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-11-10 04:25:05.360546: W tensorflow/core/common_runtime/bfc_al

...

Epoch 56/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6800 - accuracy: 0.5617 - val_loss: 0.6944 - val_accuracy: 0.4768 - lr: 0.0010
Epoch 57/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6886 - accuracy: 0.5403 - val_loss: 0.6955 - val_accuracy: 0.4768 - lr: 0.0010
Epoch 58/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6957 - accuracy: 0.4893 - val_loss: 0.6943 - val_accuracy: 0.4768 - lr: 0.0010
Epoch 59/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6664 - accuracy: 0.6005 - val_loss: 0.6923 - val_accuracy: 0.5232 - lr: 0.0010
Epoch 60/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6857 - accuracy: 0.5590 - val_loss: 0.6934 - val_accuracy: 0.5232 - lr: 0.0010
Epoch 61/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6954 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.5471 - lr: 0.0010
Epoch 62/300
1000/1000 [==========

Epoch 107/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6222 - accuracy: 0.6557 - val_loss: 0.6719 - val_accuracy: 0.5687 - lr: 0.0010
Epoch 108/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6236 - accuracy: 0.6700 - val_loss: 0.6402 - val_accuracy: 0.6345 - lr: 0.0010
Epoch 109/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.5933 - accuracy: 0.6917 - val_loss: 0.6503 - val_accuracy: 0.6228 - lr: 0.0010
Epoch 110/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.6073 - accuracy: 0.6752 - val_loss: 0.6202 - val_accuracy: 0.6481 - lr: 0.0010
Epoch 111/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.5882 - accuracy: 0.6865 - val_loss: 0.6652 - val_accuracy: 0.6496 - lr: 0.0010
Epoch 112/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.5927 - accuracy: 0.6850 - val_loss: 0.6213 - val_accuracy: 0.6479 - lr: 0.0010
Epoch 113/300
1000/1000 [===

1000/1000 [==============================] - 15s 15ms/step - loss: 0.4000 - accuracy: 0.8267 - val_loss: 0.4348 - val_accuracy: 0.7929 - lr: 0.0010
Epoch 158/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4375 - accuracy: 0.8005 - val_loss: 0.3921 - val_accuracy: 0.8211 - lr: 0.0010
Epoch 159/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.3795 - accuracy: 0.8278 - val_loss: 0.5050 - val_accuracy: 0.7787 - lr: 0.0010
Epoch 160/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.3731 - accuracy: 0.8440 - val_loss: 0.5123 - val_accuracy: 0.7613 - lr: 0.0010
Epoch 161/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4056 - accuracy: 0.8195 - val_loss: 0.5973 - val_accuracy: 0.6935 - lr: 0.0010
Epoch 162/300
1000/1000 [==============================] - 15s 15ms/step - loss: 0.4175 - accuracy: 0.8087 - val_loss: 0.4082 - val_accuracy: 0.8089 - lr: 0.0010
Epoch 163/300
1000/1000 [=================

Epoch 208/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3318 - accuracy: 0.8533 - val_loss: 0.3610 - val_accuracy: 0.8366 - lr: 0.0010
Epoch 209/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2616 - accuracy: 0.8873 - val_loss: 0.4461 - val_accuracy: 0.7862 - lr: 0.0010
Epoch 210/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2651 - accuracy: 0.8947 - val_loss: 0.4989 - val_accuracy: 0.7879 - lr: 0.0010
Epoch 211/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.3297 - accuracy: 0.8575 - val_loss: 0.3025 - val_accuracy: 0.8718 - lr: 0.0010
Epoch 212/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2574 - accuracy: 0.8950 - val_loss: 0.4232 - val_accuracy: 0.8414 - lr: 0.0010
Epoch 213/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2719 - accuracy: 0.8888 - val_loss: 0.2832 - val_accuracy: 0.8867 - lr: 0.0010
Epoch 214/300
1000/1000 [===

1000/1000 [==============================] - 14s 14ms/step - loss: 0.1924 - accuracy: 0.9275 - val_loss: 0.3390 - val_accuracy: 0.8551 - lr: 0.0010
Epoch 259/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1991 - accuracy: 0.9230 - val_loss: 0.2421 - val_accuracy: 0.9026 - lr: 0.0010
Epoch 260/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2425 - accuracy: 0.9062 - val_loss: 0.2342 - val_accuracy: 0.9079 - lr: 0.0010
Epoch 261/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.2407 - accuracy: 0.9038 - val_loss: 0.2353 - val_accuracy: 0.9052 - lr: 0.0010
Epoch 262/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1887 - accuracy: 0.9293 - val_loss: 0.2228 - val_accuracy: 0.9106 - lr: 0.0010
Epoch 263/300
1000/1000 [==============================] - 14s 14ms/step - loss: 0.1915 - accuracy: 0.9252 - val_loss: 0.2490 - val_accuracy: 0.9026 - lr: 0.0010
Epoch 264/300
1000/1000 [=================